In [14]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
import numpy as np
from PIL import Image
from glob import glob
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True)

In [38]:
def make_image_file(dataset,path):
    def to_numpy(x):
        return x.numpy()
    data = np.stack(list(dataset))
    labels = data[:,1]
    labels = list(map(to_numpy, labels))
    
    for i, (label ,arr) in tqdm(enumerate(zip(labels, data[:,0]))):
        im = Image.fromarray(np.array(arr)).resize((224,224))
        if label == 0:
            im.save(path +'0/flower_{}_{}.png'.format(label, i))
        elif label == 1:
            im.save(path +'1/flower_{}_{}.png'.format(label, i))
        elif label == 2:
            im.save(path +'2/flower_{}_{}.png'.format(label, i))
        elif label == 3:
            im.save(path +'3/flower_{}_{}.png'.format(label, i))
        elif label == 4:
            im.save(path +'4/flower_{}_{}.png'.format(label, i))

In [45]:
make_image_file(raw_train, './flower_dataset/train_images/')

2936it [00:29, 100.36it/s]


In [46]:
make_image_file(raw_validation, './flower_dataset/validation_images/')

367it [00:03, 99.54it/s] 


In [47]:
make_image_file(raw_test, './flower_dataset/test_images/')

367it [00:03, 99.04it/s] 


In [12]:
image_generator = ImageDataGenerator(
    rotation_range = 45,
    width_shift_range = 0.20,
    height_shift_range = 0.20,
    zoom_range = 0.3,
    horizontal_flip=True,
    fill_mode = 'nearest',
)

In [4]:
tf.__version__

'2.2.0'

In [5]:
data_path = glob('flower_dataset/train_images/*/*')

In [9]:
data_path[0].split('_')[-2]

'3'

In [7]:
path = './flower_dataset/augmented_data/'

In [13]:
for i in data_path:
    img = tf.io.read_file(i)
    img = tf.io.decode_image(img)
    
    img = img[tf.newaxis, ...]
    num = 0
    for batch in image_generator.flow(img, batch_size=1,save_prefix=i.split('_')[-2] ,save_to_dir=path, save_format='png'):
        num += 1
        if num > 10:
            break